In [1]:
import os
from datetime import datetime
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession, Row
from pyspark.sql.types import *
from pyspark.sql import SQLContext
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, RegressionEvaluator
from pyspark.ml.classification import RandomForestClassifier, GBTClassifier, LogisticRegression
from pyspark.ml.feature import StringIndexer, OneHotEncoder,  VectorAssembler, VectorIndexer, IndexToString
from pyspark.sql.window import Window
from pyspark.storagelevel import StorageLevel
import pyspark.sql.functions as f
from pyspark import SparkContext
from pyspark.conf import SparkConf
from pyspark.ml import Pipeline

In [2]:
pyspark_submit_args = '--packages org.mongodb.spark:mongo-spark-connector_2.11:2.4.0 pyspark-shell'
os.environ["PYSPARK_SUBMIT_ARGS"] = pyspark_submit_args

spark = SparkSession \
    .builder \
    .appName("aqi") \
    .config("spark.mongodb.input.uri", "mongodb://34.221.229.103/mydb.air")\
    .getOrCreate()
spark.conf.set("spark.executor.memory", '4g')
spark.conf.set('spark.executor.cores', '4')
spark.conf.set('spark.cores.max', '4')
spark.conf.set("spark.driver.memory",'4g')
df = spark.read.format("com.mongodb.spark.sql.DefaultSource").load()
df.printSchema()

root
 |-- Unnamed: 0.1.1: integer (nullable = true)
 |-- _id: struct (nullable = true)
 |    |-- oid: string (nullable = true)
 |-- aqi_co: double (nullable = true)
 |-- aqi_no2: double (nullable = true)
 |-- aqi_o3: double (nullable = true)
 |-- aqi_pm10: double (nullable = true)
 |-- aqi_pm25_frm: double (nullable = true)
 |-- aqi_pm25_nonfrm: double (nullable = true)
 |-- aqi_so2: double (nullable = true)
 |-- arithmetic_mean_co: double (nullable = true)
 |-- arithmetic_mean_no2: double (nullable = true)
 |-- arithmetic_mean_o3: double (nullable = true)
 |-- arithmetic_mean_pm10: double (nullable = true)
 |-- arithmetic_mean_pm25_frm: double (nullable = true)
 |-- arithmetic_mean_pm25_nonfrm: double (nullable = true)
 |-- arithmetic_mean_pm25_speciation: double (nullable = true)
 |-- arithmetic_mean_pressure: double (nullable = true)
 |-- arithmetic_mean_so2: double (nullable = true)
 |-- arithmetic_mean_temp: double (nullable = true)
 |-- arithmetic_mean_wind: double (nullable = tr

In [3]:
va = VectorAssembler(outputCol="features", inputCols=[
 'aqi_co',
 'aqi_no2',
 'aqi_o3',
 'aqi_pm10',
 'aqi_pm25_frm',
 'aqi_pm25_nonfrm',
 'aqi_so2',
 'arithmetic_mean_co',
 'arithmetic_mean_no2',
 'arithmetic_mean_o3',
 'arithmetic_mean_pm10',
 'arithmetic_mean_pm25_frm',
 'arithmetic_mean_pm25_nonfrm',
 'arithmetic_mean_pm25_speciation',
 'arithmetic_mean_pressure',
 'arithmetic_mean_so2',
 'arithmetic_mean_temp',
 'arithmetic_mean_wind',
 'county_code',
 'day',
 'dow',
 'first_max_value_co',
 'first_max_value_no2',
 'first_max_value_o3',
 'first_max_value_pm10',
 'first_max_value_pm25_frm',
 'first_max_value_pm25_nonfrm',
 'first_max_value_pm25_speciation',
 'first_max_value_pressure',
 'first_max_value_so2',
 'first_max_value_temp',
 'first_max_value_wind',
 'latitude',
 'longitude',
 'max_aqi',
 'max_aqi_before_yesterday',
 'max_aqi_yesterday',
 'month',
 'observation_count_co',
 'observation_count_no2',
 'observation_count_o3',
 'observation_count_pm10',
 'observation_count_pm25_frm',
 'observation_count_pm25_nonfrm',
 'observation_count_pm25_speciation',
 'observation_count_pressure',
 'observation_count_so2',
 'observation_count_temp',
 'observation_count_wind',
 'site_num']) 

In [4]:
df2 = va.transform(df).select("features", "label")

In [5]:
# Load and parse the data file, converting it to a DataFrame.
data = df2

# Index labels, adding metadata to the label column.
# Fit on whole dataset to include all labels in index.
labelIndexer = StringIndexer(inputCol="label", outputCol="indexedLabel").fit(data)
dataIndexed = labelIndexer.transform(data)

data = dataIndexed.drop('label').withColumnRenamed('indexedLabel','label')
# Automatically identify categorical features, and index them.
# Set maxCategories so features with > 4 distinct values are treated as continuous.
#featureIndexer =\
#    VectorIndexer(inputCol="features", outputCol="indexedFeatures").fit(data)

# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = data.randomSplit([0.7, 0.3])

In [6]:
%%time

lr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)

# Convert indexed labels back to original labels.
labelConverter = IndexToString(inputCol="prediction", outputCol="predictedLabel",
                               labels=labelIndexer.labels)

# Chain indexers and forest in a Pipeline
pipeline = Pipeline(stages=[lr, labelConverter])

# Fit the model
# Train model.  This also runs the indexers.
model = pipeline.fit(trainingData)
# Make predictions.
predictions = model.transform(testData)

# Select example rows to display.
predictions.select("predictedLabel", "label", "features").show(5)

# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g" % (1.0 - accuracy))

lrModel = model.stages[0]
print(lrModel)  # summary only

+--------------+-----+--------------------+
|predictedLabel|label|            features|
+--------------+-----+--------------------+
|          good|  0.0|[0.0,8.0,14.0,19....|
|          good|  0.0|[1.0,9.0,13.0,12....|
|          good|  1.0|[1.0,9.0,29.0,19....|
|          good|  1.0|[1.0,10.0,23.0,19...|
|          good|  1.0|[1.0,11.0,13.0,19...|
+--------------+-----+--------------------+
only showing top 5 rows

Test Error = 0.284916
LogisticRegression_4a69b230866483d28a70
CPU times: user 194 ms, sys: 148 ms, total: 341 ms
Wall time: 23min 14s
